<a href="https://colab.research.google.com/github/Andy-Lewis-Sapner/ITCC-Ant-SemB24/blob/master/Exercises/Exercise8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase
!pip install nltk

In [ ]:
import nltk
from nltk.chat.util import Chat, reflections
from firebase import firebase
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
def connect_to_DB(DB_url):
 return firebase.FirebaseApplication(DB_url, None)

def getJSONFromDB():
 return list(FBConn.get("/onShapeJSON/", None).values())[-1]

DB_url = 'https://onshapeassistant-default-rtdb.firebaseio.com/'
FBConn = connect_to_DB(DB_url)

In [ ]:
data = pd.DataFrame(getJSONFromDB())

def getUsers():
  users_list = data['User'].unique().tolist()
  response = "\n"
  for user in users_list:
    response += f"{user}\n"
  return response

def getCountsOfTopActions(user_input):
  split_user_input = user_input.split(' ')
  wanted_actions_number = 0
  for res in split_user_input:
    if res.isnumeric():
      wanted_actions_number = res
      break

  actions_dict = data['Description'].value_counts().to_dict()
  sorted_actions = sorted(actions_dict.items(), key=lambda x: x[1], reverse=True)
  wanted_actions = sorted_actions[:int(wanted_actions_number)]
  response = ""
  for action, count in wanted_actions:
    response += f"{action}: {count}\n"
  return response

def getLastDateOfProject():
  return str(data['Time'].max())

def getActionsOfAUser(user_input):
  user_input = user_input.split(' ')
  users = data['User'].unique().tolist()
  for res in user_input:
    res = res[:-1]
    if res in users:
      user_actions = data[data['User'] == res]['Description'].value_counts().to_dict()
      user_actions_mean = np.mean(list(user_actions.values()))
      response = "\n"
      for action, count in user_actions.items():
        if count > user_actions_mean:
          response += f"{action}\n"
      return response
  return "No user found"

def getDurationOfProject():
  return str(pd.to_datetime(data['Time'].max()) - pd.to_datetime(data['Time'].min()))

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatBot.', 'I go by the name ChatBot.']),
    (r'my name is (.*)', ['glad to meet you.', 'nice to meet you.']),
    (r'thank you (.*)', ['welcome!.', 'happy to help!.']),
    (r'What is the purpose of the system ?', ['The purpose of the system is to help software managers improve the teams preformance by analyzing theirs work']),
    (r'whats included in the software', ['there is a action counter aswell as graphs for every action per worker/student , there is also a notification page for the manager to see important user activity or system notifications']),
    (r'what is onshape|onshape|on shape', ['Onshape is a cloud-based CAD platform for product design and development. Learn more here: https://www.onshape.com/']),
    (r'how to filter the graph',['by clicking the desirable data']),
    (r'how to add a new team member',['at this moment of development we dont have this feature yet.']),
    (r'how to show important actions',['important actions are individual, you can filter them to your liking']),
    (r'how to show who did the most actions ',['if you go to the team graph section, you can filter it with actions and see who contributed the most']),
    (r'can you list all users?', [getUsers()]),
    (r'can you list top (.*) common actions?', [getCountsOfTopActions(user_input)]),
    (r'what is the last date of a project?', [getLastDateOfProject()]),
    (r'can you list common actions of (.*)?', [getActionsOfAUser(user_input)]),
    (r'what is the duration of the project?', [getDurationOfProject()])
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

# Start the conversation
print("\nHello! I'm ChatBot for team Ant. Do you have any questions about the json?")
while True:
    user_input = input("You: ")

    response = chatbot.respond(user_input)
    print("ChatBot:", response)
    if user_input.lower() == 'exit' or user_input.lower() == 'bye':
        break